In [125]:
import pandas as pd
import numpy as np
import xgboost as xgb

In [126]:
iteminfo_train = pd.read_csv('/Users/jerry/Desktop/udemy courses/Kaggle数据分析/Avito Duplicate Ads Detection/ItemInfo_train.csv')
itempairs_train = pd.read_csv('/Users/jerry/Desktop/udemy courses/Kaggle数据分析/Avito Duplicate Ads Detection/ItemPairs_train.csv')
location = pd.read_csv('/Users/jerry/Desktop/udemy courses/Kaggle数据分析/Avito Duplicate Ads Detection/Location.csv')
category = pd.read_csv('/Users/jerry/Desktop/udemy courses/Kaggle数据分析/Avito Duplicate Ads Detection/Category.csv')

In [127]:
iteminfo_train.head()

,itemID,categoryID,title,description,images_array,attrsJSON,price,locationID,metroID,lat,lon
0,1,81,Продам Камаз 6520,Продам Камаз 6520 20 тонн,"1064094, 5252822, 6645873, 6960145, 9230265","{""Вид техники"":""Грузовики""}",300000.0,648140,NaN,64.686946,30.815924
1,3,14,Yamaha r6,Весь в тюнинге.,"11919573, 14412228, 3204180, 6646877","{""Вид техники"":""Мотоциклы"", ""Вид мотоцикла"":""С...",300000.0,639040,NaN,55.678037,37.256548
2,4,84,iPhone 3gs 8gb,"Телефон в хорошем состоянии, трещин и сколов н...","14384831, 6102021","{""Вид телефона"":""iPhone""}",3500.0,640650,NaN,56.239398,43.460458
3,7,84,Xiaomi Mi4 3гб RAM + 16гб ROM белый,"Отличный подарок на новый год от ""китайской ap...",NaN,"{""Вид телефона"":""Другие марки""}",13500.0,662210,NaN,55.777170,37.586194
4,8,39,Лыжные ботинки,"Лыжные ботинки в хорошем состоянии, 34 размер","13718854, 4787310","{""Вид товара"":""Зимние виды спорта""}",500.0,624360,NaN,55.777170,37.586194


In [128]:
itempairs_train.head()

,itemID_1,itemID_2,isDuplicate,generationMethod
0,1,4112648,1,1
1,3,1991275,1,1
2,4,1223296,0,1
3,7,1058851,1,1
4,8,2161930,1,1


In [129]:
location.head()

,locationID,regionID
0,621551,621550
1,621552,621550
2,621553,621550
3,621554,621550
4,621555,621550


In [130]:
category.head()

,categoryID,parentCategoryID
0,9,1
1,10,1
2,11,1
3,14,1
4,19,2


In [131]:
itempairs_train1 = itempairs_train[['itemID_2','itemID_1','isDuplicate','generationMethod']]

In [132]:
itempairs_train1 = itempairs_train1.rename(columns = {'itemID_2':'itemID_1', 'itemID_1':'itemID_2'})

In [133]:
#把数据复制一下 贴到末尾处
itempairs_train2 = pd.concat([itempairs_train,itempairs_train1])
print(itempairs_train2.shape)
print(itempairs_train.shape)

(5982792, 4)
(2991396, 4)


In [134]:
itempairs_train[['itemID_1','itemID_2']]

,itemID_1,itemID_2
0,1,4112648
1,3,1991275
2,4,1223296
3,7,1058851
4,8,2161930
5,9,694103
6,12,5637025
7,12,5279740
8,15,113701
9,16,2467698


In [135]:
itempairs_train2[['itemID_1','itemID_2']]

,itemID_1,itemID_2
0,1,4112648
1,3,1991275
2,4,1223296
3,7,1058851
4,8,2161930
5,9,694103
6,12,5637025
7,12,5279740
8,15,113701
9,16,2467698


In [136]:
#用itemID 把 itempairs_train 合并到 iteminfo_train
iteminfo_train1 = iteminfo_train.merge(itempairs_train2,left_on='itemID', right_on='itemID_1' , how='left')

In [137]:
iteminfo_train1.head()

,itemID,categoryID,title,description,images_array,attrsJSON,price,locationID,metroID,lat,lon,itemID_1,itemID_2,isDuplicate,generationMethod
0,1,81,Продам Камаз 6520,Продам Камаз 6520 20 тонн,"1064094, 5252822, 6645873, 6960145, 9230265","{""Вид техники"":""Грузовики""}",300000.0,648140,NaN,64.686946,30.815924,1,4112648,1,1
1,3,14,Yamaha r6,Весь в тюнинге.,"11919573, 14412228, 3204180, 6646877","{""Вид техники"":""Мотоциклы"", ""Вид мотоцикла"":""С...",300000.0,639040,NaN,55.678037,37.256548,3,1991275,1,1
2,4,84,iPhone 3gs 8gb,"Телефон в хорошем состоянии, трещин и сколов н...","14384831, 6102021","{""Вид телефона"":""iPhone""}",3500.0,640650,NaN,56.239398,43.460458,4,1223296,0,1
3,7,84,Xiaomi Mi4 3гб RAM + 16гб ROM белый,"Отличный подарок на новый год от ""китайской ap...",NaN,"{""Вид телефона"":""Другие марки""}",13500.0,662210,NaN,55.777170,37.586194,7,1058851,1,1
4,8,39,Лыжные ботинки,"Лыжные ботинки в хорошем состоянии, 34 размер","13718854, 4787310","{""Вид товара"":""Зимние виды спорта""}",500.0,624360,NaN,55.777170,37.586194,8,2161930,1,1


In [138]:
#去除多余数据
iteminfo_train1 = iteminfo_train1.drop(['itemID_1','itemID_2'],axis=1) #axis=1意味着选择column 而不是 index

In [139]:
iteminfo_train2 = iteminfo_train1.merge(location,on='locationID', how='left')
train = iteminfo_train2.merge(category,on='categoryID', how='left')

In [140]:
train.head()

,itemID,categoryID,title,description,images_array,attrsJSON,price,locationID,metroID,lat,lon,isDuplicate,generationMethod,regionID,parentCategoryID
0,1,81,Продам Камаз 6520,Продам Камаз 6520 20 тонн,"1064094, 5252822, 6645873, 6960145, 9230265","{""Вид техники"":""Грузовики""}",300000.0,648140,NaN,64.686946,30.815924,1,1,648070,1
1,3,14,Yamaha r6,Весь в тюнинге.,"11919573, 14412228, 3204180, 6646877","{""Вид техники"":""Мотоциклы"", ""Вид мотоцикла"":""С...",300000.0,639040,NaN,55.678037,37.256548,1,1,637680,1
2,4,84,iPhone 3gs 8gb,"Телефон в хорошем состоянии, трещин и сколов н...","14384831, 6102021","{""Вид телефона"":""iPhone""}",3500.0,640650,NaN,56.239398,43.460458,0,1,640310,6
3,7,84,Xiaomi Mi4 3гб RAM + 16гб ROM белый,"Отличный подарок на новый год от ""китайской ap...",NaN,"{""Вид телефона"":""Другие марки""}",13500.0,662210,NaN,55.777170,37.586194,1,1,662000,6
4,8,39,Лыжные ботинки,"Лыжные ботинки в хорошем состоянии, 34 размер","13718854, 4787310","{""Вид товара"":""Зимние виды спорта""}",500.0,624360,NaN,55.777170,37.586194,1,1,624300,7


train.columns

In [142]:
train.to_csv('/Users/jerry/Desktop/udemy courses/Kaggle数据分析/Avito Duplicate Ads Detection/train.csv')